In [1]:
import torch 
import torch.nn as nn 
import numpy as np
from collections import OrderedDict

from utils.losses import protop_loss
from utils.sample_parameters import SingleGenerator, HiddenAndKernelGenerator, MonotonicGenerator
from utils.sample_parameters import ParamGenerators
from models.ProtICU import ProtICU
from utils.train_n_test import TrainTest

In [2]:
train, val, test = torch.load('data/in-hospital-mortality/tensors.pkl')

In [3]:
# Calculating class weights
perc_mort = np.concatenate((train[1], val[1], test[1])).mean()
class_weights = torch.Tensor([perc_mort, 1-perc_mort])

In [514]:
generators_dict = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': SingleGenerator([128, 256, 512]),
    'EPOCHS': SingleGenerator([20, 40, 50]),
    'OPTIMIZER': SingleGenerator([torch.optim.Adam]),
    'LEARNING_RATE': SingleGenerator([1e-5, 5e-5, 1e-4, 5e-4]),
    'LOSS': SingleGenerator([protop_loss(class_weights, .5 , .5), protop_loss(class_weights, 1, 1),
                             protop_loss(class_weights, 1 , .5),  protop_loss(class_weights, 2, 1)]),
    'EARLY_STOPPING': SingleGenerator([True, False]),
    'PATIENCE': SingleGenerator(list(range(2,4))),
    'MIN_DELTA': SingleGenerator([5e-5, 1e-4, 5e-4, 1e-3, 5e-3]), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_AND_KERNEL_SIZES': HiddenAndKernelGenerator(range(2,5), [64, 128, 256, 512], [3, 5, 7, 9], 
                               ascending=(True, False)),
    'MAXPOOL': SingleGenerator([2]),
    'OBO_SIZES': MonotonicGenerator(range(1,3), [256, 512], ascending=False),
    'PROTOTYPE_NUM': SingleGenerator([10,20,30]),
    'DROPOUT': SingleGenerator(np.arange(2)/10), 
    'PROTO_ACTIVATION': SingleGenerator(['log'])
} # each of these params are sampled INDEPENDENTLY of one another

In [538]:
class_weights[0] * 21139

tensor(2797.)

In [541]:
val[0].shape

torch.Size([2114, 48, 46])

In [535]:
train[0].shape[0] + val[0].shape[0] + test[0].shape[0]

21139

In [515]:
fixed_gen = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': SingleGenerator([256]),
    'EPOCHS': SingleGenerator([20]),
    'OPTIMIZER': SingleGenerator([torch.optim.Adam]),
    'LEARNING_RATE': SingleGenerator([1e-5]),
    'LOSS': SingleGenerator([protop_loss(class_weights, 1, 1)]),
    'EARLY_STOPPING': SingleGenerator([True]),
    'PATIENCE': SingleGenerator(list(range(1,2))),
    'MIN_DELTA': SingleGenerator([1e-4]), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_AND_KERNEL_SIZES': HiddenAndKernelGenerator(range(1,2), [256], [5], 
                               ascending=(True, False)),
    'MAXPOOL': SingleGenerator([2]),
    'OBO_SIZES': MonotonicGenerator(range(1,2), [128], ascending=False),
    'PROTOTYPE_NUM': SingleGenerator([20]),
    'DROPOUT': SingleGenerator([0.2]),
    'PROTO_ACTIVATION': SingleGenerator(['log'])
}

In [516]:
N = 12
gen = ParamGenerators(generators_dict)
#gen = ParamGenerators(fixed_gen)
param_samples = gen.sample(N)

In [517]:
%%time
stats = []
for i in np.arange(N):
    print(param_samples[i])
    param_samples[i]['EPOCHS']
    push_epochs = [param_samples[0]['EPOCHS'] -1] #list(range(5,param_samples[0]['EPOCHS'],2))
    run = TrainTest(ProtICU, (train, val, test), param_samples[i], push_epochs=push_epochs)
    run.train()
    stats.append(run.test())
    stats[i]['prototype_raw'] = run.prototype_raw
    stats[i]['model'] = run.model

  0%|          | 0/115 [00:00<?, ?it/s]

{'BATCH_SIZE': 128, 'EPOCHS': 50, 'OPTIMIZER': <class 'torch.optim.adam.Adam'>, 'LEARNING_RATE': 5e-05, 'LOSS': protop_loss_0.5_0.5, 'EARLY_STOPPING': False, 'PATIENCE': 2, 'MIN_DELTA': 0.001, 'MAXPOOL': 2, 'OBO_SIZES': array([256]), 'PROTOTYPE_NUM': 20, 'DROPOUT': 0.1, 'PROTO_ACTIVATION': 'log', 'HIDDEN_SIZES': array([256, 512]), 'KERNEL_SIZES': array([5, 3])}


  0%|          | 0/115 [00:00<?, ?it/s]

Epoch: 0, train_loss: -0.2549084722995758, valid_loss: -0.678093433380127


  0%|          | 0/115 [00:00<?, ?it/s]

...

Epoch: 17, train_loss: -2.3471434116363525, valid_loss: -2.2942981719970703


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch: 18, train_loss: -2.3525094985961914, valid_loss: -2.3035287857055664


100%|██████████| 8/8 [00:00<00:00,  8.52it/s]


Epoch: 19, train_loss: -2.367727279663086, valid_loss: -2.330211639404297
CPU times: user 12h 29min 20s, sys: 6h 47min 9s, total: 19h 16min 30s
Wall time: 4h 47min 33s


In [518]:
# make params_sample
for i in np.arange(N):
    for key,val in stats[i].items():
        param_samples[i][key] = val
        
# saving 
torch.save(param_samples, 'results/Proto_experiment_push_last_N'+str(N)+'.pkl')

# getting pd 
import pandas as pd 
df = pd.DataFrame(param_samples)

## Visualising Tensors 

In [520]:
selected = [d for d in param_samples if d['PROTO_ACTIVATION'] == 'log' and 
              not d['EARLY_STOPPING'] and 
              d['auroc'] > 0.75]

In [369]:
%load_ext autoreload
%autoreload 2
from utils.visuals import compare_with_prototypes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [370]:
train, val, test = torch.load('data/in-hospital-mortality/tensors.pkl')
cols = torch.load('data/in-hospital-mortality/tensor_features.pkl')

# INDEXES 
idx_dict = {
    'CONT': np.arange(14), 
    'CONT_MASK': np.arange(29, 43), 
    'DISC': [np.arange(14, 18), np.arange(18, 24), np.arange(24, 29)],
    'DISC_MASK': np.arange(43, 46)
}
# DISCRETE MAPPING
disc_orders = [
    ['no_response', 'to_pain', 'to_speech', 'spontaneously'], # eyes
    ['no_response', 'abnormal_extension', 'abnormal_flexion', 
              'flex-withdraws', 'localizes_pain', 'obeys_commands'], # motor 
    ['no_response', 'incomprehensible_sounds', 
               'inappropriate_words', 'confused', 'oriented']] # verbal

# SORT DISC ACCORDING TO DISC_ORDERS
import re 
cols = np.array([re.sub('^[^_]*_', '', s) for s in list(cols)] )
new_disc = []
for i,r in enumerate(idx_dict['DISC']): 
    d = dict(zip(list(cols[r]), r))
    temp = [d[j] for j in disc_orders[i]]
    new_disc.append(np.array(temp))
idx_dict['DISC'] = new_disc

In [523]:
model_idx = 0
model = selected[model_idx]['model']
patch_size = np.power(2, len(selected[model_idx]['HIDDEN_SIZES']))
print(selected[model_idx]['auroc'])
print(selected[model_idx]['LOSS'])

check = model(test[0][:200], get_patch=True)
dead = np.arange(200)[(check[0] > .5)[:,1]]
j = 0 

0.75093389452169
protop_loss_0.5_0.5


In [ ]:
%%time
j += 1
i = dead[j]
prot_num = 3
print(i)

# DATA 
data = test[0][i:i+1]
result = model(data, get_patch=True)
most_activated_proto = torch.topk(result[1], prot_num, largest=False).indices.squeeze().numpy()
print(most_activated_proto)
patch_idx = result[2].squeeze()[most_activated_proto].numpy()
highlight_data = [(pi * patch_size, (pi + 1) * patch_size) for pi in patch_idx]

# PROTOTYPE
prototypes = [selected[model_idx]['prototype_raw'][0][i] for i in most_activated_proto]
patch_idx = [int(selected[model_idx]['prototype_raw'][1][i]) for i in most_activated_proto]
highlight_proto = [(pi * patch_size, (pi + 1) * patch_size) for pi in patch_idx]

highlights = [[highlight_data[i], highlight_proto[i]] for i in range(len(highlight_data))]

compare_with_prototypes(data.squeeze(), prototypes, highlights, cols, idx_dict, 
                        exclude=['Height', 'Weight', 'Capillary refill rate',
                                 'Fraction inspired oxygen', 'Glascow coma scale eye opening',
                                 'pH']) #, save='results/visuals/model_0_stroke.jpg')